In [1]:
import itertools
import numpy as np
import matplotlib.pyplot as plt

import hoomd
import gsd.hoomd
import freud
import coxeter

#### creating unit volume particles

In [2]:
alpha=1.0
family323 = coxeter.families.Family323Plus()
particle = family323.get_shape(a=alpha, c=0.2*alpha+0.8)
verts = particle.vertices/particle.volume**(1/3)

new_particle = coxeter.shapes.ConvexPolyhedron(verts)
new_particle.volume

1.0

#### target crystal

In [3]:
uc = freud.data.UnitCell.bcc()

box, position = uc.generate_system(num_replicas=4, scale=2)
position+=0.5

N_particles = len(position)

#### creating snapshot

In [4]:
snapshot = gsd.hoomd.Frame()
snapshot.particles.types = ['A']
snapshot.particles.N = N_particles
snapshot.particles.typeid = [0] * N_particles
snapshot.particles.position = position[:N_particles]
snapshot.particles.orientation = [1, 0, 0, 0]*N_particles

verts = verts.tolist()
snapshot.particles.type_shapes = [{'type': 'ConvexPolyhedron',
                                   'vertices': verts}]

snapshot.configuration.box = [box.Lx, box.Ly, box.Lz, box.xy, box.xz, box.yz]

#### save snapshot

In [5]:
with gsd.hoomd.open(name='./DATA/lattice.gsd', mode='w') as f:
    f.append(snapshot)